# Automação Web e Busca de Informações com Python



Pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados e depois criar e exportar um novo arquivo excel com os valores atualizados

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing



In [1]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
navegador = webdriver.Chrome(r'C:\Users\Home\anaconda3\chromedriver_win32\chromedriver') 

#pegar cotacao dolar
navegador.get('https://www.google.com.br/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação dólar")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar= navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")


#pegar cotacaco do euro
navegador.get('https://www.google.com.br/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro= navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")


#pegar cotacaco do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro= navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace("," , ".")


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [2]:
import pandas as pd
tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [3]:
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)
tabela["Preço de Compra"] = tabela["Preço Base Original"] * tabela["Cotação"]
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"] 
tabela["Preço de Venda"] = tabela["Preço de Venda"].map("{:.2f}".format)
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final,Preço de Compra,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.226000,4999.95,1.40,6999.930,5225.947740,7316.33
1,Carro Renault,4500.00,Euro,6.136181,27000.00,2.00,54000.000,27612.816723,55225.63
2,Notebook Dell,899.99,Dólar,5.226000,4499.95,1.70,7649.915,4703.347740,7995.69
3,IPhone,799.00,Dólar,5.226000,3995.00,1.70,6791.500,4175.574000,7098.48
4,Carro Fiat,3000.00,Euro,6.136181,18000.00,1.90,34200.000,18408.544482,34976.23
5,Celular Xiaomi,480.48,Dólar,5.226000,2402.40,2.00,4804.800,2510.988480,5021.98
6,Joia 20g,20.00,Ouro,294.740000,7000.00,1.15,8050.000,5894.800000,6779.02


### Agora vamos exportar a nova base de preços atualizada

In [4]:
tabela.to_excel("Produtos Novo.xlsx", index=False)